In [1]:
import torch
import torch.nn as nn

### Convolution Blocks

In [2]:
def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)

In [3]:
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

### Basic Block, Bottle neck

In [4]:
class BasicBlock(nn.Module):
    expansion=1

    def __init__(self, in_channels, out_channels, stride=1, down_sample=None, groups=1, base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('Basic Block only supports groups=1, base_width=64')
        if dilation > 1:
            raise NotImplementedError('Dilation > 1 not supported in Basic Block')

        # Basic Block 구조
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = norm_layer(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = norm_layer(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.stride = stride


    @staticmethod
    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # down sample
        if self.down_sample is not None:
            identity = self.down_sample(x)

        # relu 를 거치면 값이 한개 밖에 남지 않는다.
        out += identity
        out = self.relu(out)

        return out

In [5]:
class BottleNeck(nn.Module):
    expansion = 4


    def __init__(self, in_channels, out_channels, stride=1, down_sample=None, groups=1, base_width=64, dilation=1, norm_layer=None):
        super(BottleNeck, self).__init__()
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        # ResNext 나 WideResNet 의 경우 사용
        width = int(out_channels * (base_width / 64.)) * groups

        # BottleNeck 의 구조
        self.conv1 = conv1x1(in_channels, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation) # No matter use down_sample elsewhere
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, out_channels * self.expansion)
        self.bn3 = norm_layer(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.down_sample = down_sample
        self.stride = stride


    @staticmethod
    def forward(self, x):
        identity = x

        # 1x1 convolution layer
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # 3x3 convolution layer
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        # 1x1 convolution layer
        out = self.conv3(out)
        out = self.bn3(out)

        # skip connection
        if self.down_sample is not None:
            identity = self.down_sample(x)

        out += identity
        out = self.relu(out)

        return out

### ResNet

In [6]:
class ResNet(nn.Module):


    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False, groups=1, width_per_group=64, replace_stride_width_dilation=None, norm_layer=None):
        super(ResNet, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self.norm_layer = norm_layer

        # default values
        self.in_channels = 64
        self.dilation = 1

        if replace_stride_width_dilation is None:
            replace_stride_width_dilation = [False, False, False]
        if len(replace_stride_width_dilation) != 3:
            raise ValueError("replace_stride_width_dilation should be None" "or a 3-element tuple, got {}".format(replace_stride_width_dilation))
        self.groups = groups
        self.base_width = width_per_group

        # 처음 입력으로 RGB 이미지를 사용하기 때문에 input_channel = 3
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2 # 여기서 부터 down_sample 적용
                                       ,dilate=replace_stride_width_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_width_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_width_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weiht, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BottleNeck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)


    def _make_layer(self, block, out_channels, blocks, stride=1, dilate=False):

        """
        - block: block 종류 지정
        - in_channels: feature map size
        - blocks: 블록 갯수 지정
        - stride 와 dilate 고정
        """

        norm_layer = self._norm_layer
        down_sample = None
        previous_dilation = self.dilation

        if dilate:
            self.dilation *= stride
            stride = 1

        if stride != 1 or self.in_channels != out_channels * block.expansion:
            down_sample = nn.Sequential(
                conv1x1(self.in_channels, out_channels * block.expansion, stride),
                norm_layer(out_channels * block.expansion)
            )

        layers = []
        # 블록 내 시작 layer, down_sample 필요
        layers.append(block(self.in_channels, out_channels, stride, down_sample, self.groups, self.base_width, previous_dilation, norm_layer))
        self.in_channels = out_channels * block.expansion

        # 블록 반복
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer = norm_layer))

        return nn.Sequential(*layers)


    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


    def forward(self, x):
        return self._forward_impl(x)